## Graph Attention Network (GAT)

### Importing Dependencies

We import the necessary libraries and functions, ensuring that all required modules and helper functions are properly integrated.


In [2]:
import networkx as nx
from torch_geometric.utils import from_networkx
import os
import sys

# gat → models → src
src_path = os.path.abspath(os.path.join(os.getcwd(), "..", ".."))
if src_path not in sys.path:
    sys.path.append(src_path)


import import_ipynb 
from utils.wrapper.networkx_to_pyg import networkx_to_pyg
from utils.add_dummy_node_features import add_dummy_node_features



### Loading and Preparing Graph Data from GraphML Files

This code snippet loads a series of bicycle traffic network graphs stored in GraphML format and prepares them for training with PyTorch Geometric (PyG). The objective is to convert each monthly graph into a format compatible with Graph Neural Networks (GNNs), ensuring that edge features are retained.

Each NetworkX graph is converted into a PyG `Data` object using a custom helper function `networkx_to_pyg`. This function ensures that essential edge attributes such as:

- `tracks` (the number of bicycles traveling from the starting to the ending point),
- `month` and `year`,
- `speed_rel` (relative speed),

are preserved during the conversion process.

PyG expects data in a specific structure, particularly when edge attributes are used in models like GATv2.

`data_list` contains multiple `torch_geometric.data.Data` objects, each representing a graph.


In [3]:

# Initialize an empty list to store the PyTorch Geometric Data objects
data_list = []

# Iterate over the 12 graph files (from 0 to 11)
for i in range(12):  # 0 to 11
    # Build the path to the graph file
    path = f"../../../graphs/2023/bike_network_2023_{i}.graphml"
    
    # Read the graph from the GraphML file
    G_nx = nx.read_graphml(path)
    
    # Ensure the graph is loaded as a directed graph (DiGraph)
    G_nx = nx.DiGraph(G_nx)
    
    # Use the custom function to convert the NetworkX graph to a PyTorch Geometric Data object
    # The edge attributes (such as 'tracks', 'month', 'year', 'speed_rel') will be preserved
    data = networkx_to_pyg(G_nx)
    
    # Append the Data object to the list
    data_list.append(data)

# Check the result - number of graphs (Data objects) loaded
print(f"Number of graphs: {len(data_list)}")


Number of graphs: 12


### Adding Dummy Node Features to the Graphs

In this section of the code, we add **dummy node features** to our graphs. This process ensures that each node in our graphs has a **feature dimension**, even if no node features were originally present. This is an important step in preparing the data for use in Graph Neural Networks (GNNs).

**NOTE:** At a later stage, once we have implemented feature engineering, we will replace the dummy features with the engineered ones.


In [3]:
data_list = add_dummy_node_features(data_list, feature_dim=1, value=1.0)


## Implementing the GATv2 Model

We implement the **Graph Attention Network (GATv2)** model, an advanced model for Graph Neural Networks (GNNs) that is based on the principles of **attention mechanisms**. It is specifically designed to aggregate node features while learning the relationships between nodes, taking into account **edge attributes**.

The GATv2 model expects the following **input**:
- `x`: Node features → `data.x`
- `edge_index`: Edge list → `data.edge_index`
- `edge_attr`: Edge attributes → `data.edge_attr`

These values are passed from the `Data` object when calling your model.

**Output**: The model currently returns node representations (node embeddings) – a tensor with one row per node and one column per feature at the output.


In [ ]:
import torch
import torch.nn.functional as F
from torch_geometric.nn import GATv2Conv

class GATv2(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels, edge_dim, heads=1):
        super(GATv2, self).__init__()

        # First GATv2 layer, with edge attributes
        self.gat1 = GATv2Conv(in_channels, hidden_channels, heads=heads, edge_dim=edge_dim)

        # Second GATv2 layer, output dimension = out_channels
        self.gat2 = GATv2Conv(hidden_channels * heads, out_channels, heads=1, edge_dim=edge_dim)

    def forward(self, x, edge_index, edge_attr):
        # Apply first GATv2 layer with edge attributes
        x = self.gat1(x, edge_index, edge_attr)
        x = F.elu(x)

        # Apply second GATv2 layer
        x = self.gat2(x, edge_index, edge_attr)
        return x
